sem 3

Единственная задача: обобщить написанный на семинаре код и сымитировать работу одного большого отдела ABBYY, в котором есть три маленьких подотдела с лингвистами, программистами и комплингом. То есть, что у нас должно быть реализовано:


родительский класс "работник"


базовые классы "лингвист", "программист" и "компьютерный лингвист"


у всех методы work


классы "босс_лингвист", "босс_программист" и "босс_кл", которые могут наследовать (с подмешиванием) от общего класса "босс"


у боссов в атрибутах сидят их подчиненные


босс подотдела получает квесты от менеджера главного отдела и принуждает сотрудников работать


в главном отделе есть метод для выдачи квестов


соответственно, используем как наследование, так и композицию с делегированием

In [3]:
class Employee:

    def __init__(self, name, surname):
        self.name = name
        self.surname = surname
        self.__salary = 350  # _Employee__salary
        self.bankaccount = 0

    def work(self, *args):
        raise NotImplementedError

class Linguist(Employee):
    def __init__(self, name, surname):
        Employee.__init__(self, name, surname)
        #self.__salary = self._Employee__salary * 1.5
        #self.publications = []
    
    def work(self):
        print('Лингвист working...')
     
    # def readlingpapers(self, lingpaper):
    #     print('Reading...')
    #     print(f'{self.name} {self.surname} has read {lingpaper}')

class Programmer(Employee):
    def __init__(self, name, surname):
        Employee.__init__(self, name, surname)
        #self.__salary *= self._Employee__salary * 2.5
        #self.projects = []

    def work(self):
        print('Программист working...')
        
    # def learnpython(self):
    #     print('Studying...')
    #     print(f'{self.name} {self.surname} has learned a bit of Python')

class ComputerLinguist(Linguist, Programmer):
    
    def __init__(self, name, surname):
        Linguist.__init__(self, name, surname) 
        Programmer.__init__(self, name, surname)
      #  self.__salary *= self._Employee__salary * 2.9

    def work(self):
        print('Компьютерный лингвист working...')

class Boss:
    def __init__(self, subordinates):
        self.subordinates = subordinates

    def assign_task(self, task):
        for subordinate in self.subordinates:
            subordinate.work()

class LinguistBoss(Boss):
    def __init__(self, subordinates):
        super().__init__(subordinates)

class ProgrammerBoss(Boss):
    def __init__(self, subordinates):
        super().__init__(subordinates)

class ComputerLinguistBoss(Boss):
    def __init__(self, subordinates):
        super().__init__(subordinates)

class MainDepartment:
    def __init__(self, linguists, programmers, computer_linguists):
        self.linguist_boss = LinguistBoss(linguists)
        self.programmer_boss = ProgrammerBoss(programmers)
        self.computer_linguist_boss = ComputerLinguistBoss(computer_linguists)

    def assign_tasks(self):
        task = 'Выполнить task'
        self.linguist_boss.assign_task(task)
        self.programmer_boss.assign_task(task)
        self.computer_linguist_boss.assign_task(task)

linguist1 = Linguist('Vasya', 'Petrov')
linguist2 = Linguist('Petya', 'Vasilev')
programmer1 = Programmer('Vanya', 'Andreev')
programmer2 = Programmer('Andrey', 'Ivanov')
computer_linguist1 = ComputerLinguist('Roma', 'Antonov')
computer_linguist2 = ComputerLinguist('Anton', 'Romanov')

main_department = MainDepartment([linguist1, linguist2], [programmer1, programmer2], [computer_linguist1, computer_linguist2])
main_department.assign_tasks()

Лингвист working...
Лингвист working...
Программист working...
Программист working...
Компьютерный лингвист working...
Компьютерный лингвист working...


ps. я очень сильно сократила код

hw 3

Питон умеет работать со звуком, распознаванием речи и Text-To-Speech: для этого есть (весьма простые) библиотеки playsound, gTTS и Speech Recognition. Попробуйте разобраться с этими библиотеками и с их помощью создать своего собственного мини-чатбота - помощника по изучению языка: на сервер загружать его не нужно, достаточно, чтобы он работал из консоли. Чатбот должен быть написан в классах, разумеется; он должен уметь по просьбе человека переводить и озвучивать что-то на английский (можно другой или предоставлять выбор), зачитывать по просьбе человека текст на этом языке (отвечать на вопрос "как читается...?"), а также человек с его помощью может тренироваться произносить что-то вслух на изучаемом языке: человек произносит, бот слушает и сверяет распознанное с исходным текстом.


Если сделаете графический интерфейс, еще +10 баллов.

In [4]:
import sys
from PyQt5.QtWidgets import QApplication, QWidget, QVBoxLayout, QLabel, QLineEdit, QPushButton
from gtts import gTTS
from playsound import playsound
import speech_recognition as sr

In [5]:
class ChatBot(QWidget):
    def __init__(self):
        super().__init__()
        self.initUI()

    def initUI(self):
        self.setGeometry(100, 100, 500, 200)
        self.setWindowTitle('Нано-чатбот для чтения и перевода')
        
        self.layout = QVBoxLayout()
        
        self.questionLabel = QLabel('Введите текст:')
        self.layout.addWidget(self.questionLabel)

        self.questionLineEdit = QLineEdit()
        self.layout.addWidget(self.questionLineEdit)
        
        self.translateButton = QPushButton('Перевести на английский')
        self.translateButton.clicked.connect(self.translateToEnglish)
        self.layout.addWidget(self.translateButton)
        
        self.speechButton = QPushButton('Зачитать на английском')
        self.speechButton.clicked.connect(self.speakEnglish)
        self.layout.addWidget(self.speechButton)
        
        self.listenButton = QPushButton('Слушать и распознавать')
        self.listenButton.clicked.connect(self.listenAndRecognize)
        self.layout.addWidget(self.listenButton)
        
        self.resultLabel = QLabel()
        self.layout.addWidget(self.resultLabel)

        self.setLayout(self.layout)
    
    def translateToEnglish(self):
        text = self.questionLineEdit.text()
        tts = gTTS(text, lang='en')
        tts.save('translation.mp3')
        self.resultLabel.setText('Текст переведен и сохранен в translation.mp3.')

    def speakEnglish(self):
        text = self.questionLineEdit.text()
        tts = gTTS(text, lang='en')
        tts.save('speech.mp3')
        self.resultLabel.setText('Текст зачитан и сохранен в speech.mp3.')
    
    def listenAndRecognize(self):
        recognizer = sr.Recognizer()
        with sr.Microphone() as source:
            audio = recognizer.listen(source)

        try:
            recognized_text = recognizer.recognize_google(audio, language='en-US')
            self.resultLabel.setText(f'Распознанный текст: {recognized_text}')
        except sr.UnknownValueError:
            self.resultLabel.setText('Не удалось распознать речь(((')
        except sr.RequestError:
            self.resultLabel.setText('Не удалось получить результат из службы распознавания речи(((')

if __name__ == '__main__':
    app = QApplication(sys.argv)
    chatbot = ChatBot()
    chatbot.show()
    sys.exit(app.exec_())

SystemExit: 0

C:\Users\mkuzo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\IPython\core\interactiveshell.py:3513: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
